<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/notebooks/rag_x_phi3_financebenchQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture --no-stderr
%pip install langchain-nomic langchain langchain-core langchain-community chromadb --quiet
%pip install sentence_transformers FlagEmbedding --quiet

In [2]:
# LlamaCpp x GPU usage
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 MB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.78-cp310-cp310-linux_x86_64.whl size=169130738 sha256=0383155206e400bb73b9cf8a0d3ccbb216551fc68e7fed3eb1d1ae679991ec1c
  Stored in directory: /root/.cache/pip/wheels/fd/c5/bd/3b1c20081bd71ce9d28b562573c97915c790bf1ef231879a61
Successfully built llama-cpp-python


In [3]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from langchain_community.llms import LlamaCpp
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_core.prompts import PromptTemplate

import chromadb
from langchain.storage.file_system import LocalFileStore
from langchain.storage._lc_store import create_kv_docstore
from langchain.vectorstores import Chroma

from FlagEmbedding import FlagReranker

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

from tqdm import tqdm
import pandas as pd
import os

In [5]:
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-gguf Phi-3-mini-4k-instruct-fp16.gguf --local-dir ./models --local-dir-use-symlinks False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Phi-3-mini-4k-instruct-fp16.gguf: 100% 7.64G/7.64G [05:17<00:00, 24.1MB/s]
Download complete. Moving file to models/Phi-3-mini-4k-instruct-fp16.gguf
models/Phi-3-mini-4k-instruct-fp16.gguf


In [6]:
TEMP = 0
N_CTX = 4096
N_GPU_L = -1

llm_phi3 = LlamaCpp(
    model_path="/content/models/Phi-3-mini-4k-instruct-fp16.gguf",
    temperature=TEMP,
    n_ctx=N_CTX,
    n_gpu_layers = N_GPU_L,
    verbose=True
)

llama_model_loader: loaded meta data with 23 key-value pairs and 195 tensors from /content/models/Phi-3-mini-4k-instruct-fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.head_count

In [7]:
questions = pd.read_json('https://raw.githubusercontent.com/patronus-ai/financebench/main/data/financebench_open_source.jsonl', lines=True)
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   financebench_id       150 non-null    object
 1   company               150 non-null    object
 2   doc_name              150 non-null    object
 3   question_type         150 non-null    object
 4   question_reasoning    100 non-null    object
 5   domain_question_num   50 non-null     object
 6   question              150 non-null    object
 7   answer                150 non-null    object
 8   justification         100 non-null    object
 9   dataset_subset_label  150 non-null    object
 10  evidence              150 non-null    object
dtypes: object(11)
memory usage: 13.0+ KB


In [8]:
model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'}, #gpu
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
reranker = FlagReranker('BAAI/bge-reranker-large', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [19]:
### Metadata company name
prompt_metadata = PromptTemplate(
template="""
  <|assistant|> You need to identify which of the companies from the metadata list is mentioned in the users input.
  Database metadata with company names: {metadata_list}.
  Format your response as a JSON object with only a single key 'company', without any additional commentary or explanations.
  You do not need to try to answer the question itself.
  <|user|>Users input: {input}<|end|>
  <|assistant|>
""",
input_variables=["input", "metadata_list"])

retrieval_metadata = prompt_metadata | llm_phi3 | JsonOutputParser()

In [20]:
persistent_client = chromadb.PersistentClient('/content/drive/MyDrive/Thesis/chromadb')
collection = persistent_client.get_or_create_collection("reports_l2")
fs = LocalFileStore('/content/drive/MyDrive/Thesis/reports_store_location')
store = create_kv_docstore(fs)
vectorstore = Chroma(client = persistent_client,
                     collection_name="reports_l2",
                     embedding_function=bge_embeddings,
                     persist_directory='/content/drive/MyDrive/Thesis/chromadb')
vectorstore.persist()

In [21]:
metadata = vectorstore.get()['metadatas']
metadata_list = []
for i in range(len(metadata)):
  metadata_list.append(metadata[i]['company'])
metadata_list = list(set(metadata_list))

In [13]:
### Retrieval Grader
llm_retrieval = llm_phi3

prompt_retrieval_grader = PromptTemplate(
    template="""<|assistant|> You are a grader assessing relevance of a retrieved document to an evidence text.
    If the retrieved document contains the same information as an evidence text, grade it as relevant. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the evidence text.<|end|>
    <|user|> Here is the retrieved document: {document}\n Here is the evidence text: {evidence_text} <|end|>
    <|assistant|>
    """,
    input_variables=["evidence_text", "document"],
)

retrieval_grader = prompt_retrieval_grader | llm_retrieval | StrOutputParser()

In [14]:
### Generate
llm_generate = llm_phi3

prompt_generate = PromptTemplate(
    template="""<|assistant|> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know. Keep the answer concise <|end|>
    <|user|> Question: {question}. \n Context: {documents} \n Answer: <|end|>
    <|assistant|>""",
    input_variables=["question", "documents"],
)

rag_chain = prompt_generate | llm_generate | StrOutputParser()

In [15]:
### Hallucination Grader
llm_hallucination_grader = llm_phi3

# Prompt
prompt_hallucination_grader = PromptTemplate(
    template=""" <|assistant|> You are a grader assessing whether an answer is grounded in / supported by a set of facts.
    Give a binary 'yes' or 'no' score to indicate whether the answer is grounded in / supported by a set of facts.<|end|>
    <|user|> Here are the facts: {documents} \n Here is the answer: {generation}  <|end|>
    <|assistant|>""",
    input_variables=["generation", "documents"],
)

hallucination_grader = prompt_hallucination_grader | llm_hallucination_grader | StrOutputParser()

In [16]:
### Answer Grader
llm_answer_grader = llm_phi3

# Prompt
prompt_answer_grader = PromptTemplate(
    template="""<|assistant|> You are a grader assessing whether a generated answer is correct or incorrect,
    assecing it with the ground truth.
    Give a binary score 'yes' or 'no' to indicate whether the generated answer equal to or contains the ground truth.<|end|>
    <|user|> Here is the answer: {generation} \n Here is the ground truth: {truth} <|end|>
    <|assistant|>""",
    input_variables=["generation", "truth"],
)

answer_grader = prompt_answer_grader | llm_answer_grader | StrOutputParser()

In [22]:
NUM_PAR_CHUNKS = 20
N_DOCS_RETURN = 2

results_list = []

for i in tqdm(range(len(questions))):
    query = questions['question'][i]
    company = retrieval_metadata.invoke({"input": questions['company'][i], "metadata_list": metadata_list})

    parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
    child_splitter = RecursiveCharacterTextSplitter(chunk_size=256)
    big_chunks_retriever = ParentDocumentRetriever(
      vectorstore=vectorstore, docstore=store, child_splitter=child_splitter, parent_splitter=parent_splitter,
      search_kwargs={'filter': {'company': company['company']}, 'k': NUM_PAR_CHUNKS})

    passage = big_chunks_retriever.invoke(query)
    texts = []
    for i in range(len(passage)):
      texts.append([query, passage[i].page_content])
    try:
      if not texts:
        raise ValueError('Texts list is empty')
      scores = reranker.compute_score(texts)
      combined = list(zip(texts, scores))
      sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)
      top_texts = [item[0] for item in sorted_combined[:N_DOCS_RETURN]]
      docs = [inner_list[1] for inner_list in top_texts if len(inner_list)>1]

      retrieval_grade = retrieval_grader.invoke({"evidence_text": questions['evidence'][i], "document": docs})
      generation = rag_chain.invoke({"documents": docs, "question": query})
      hallucination_grade = hallucination_grader.invoke({"documents": docs, "generation": generation})
      answer_grade = answer_grader.invoke({"truth": questions['answer'][i], "generation": generation})

      results_list.append(pd.DataFrame({
            'question': [query],
            'response': [generation],
            'context': [docs],
            'retrieval_grade': [retrieval_grade],
            'hallucination_grade': [hallucination_grade],
            'answer_grade': [answer_grade]
        }))

      results = pd.concat(results_list, ignore_index=True)
      results.to_json(f'/content/drive/MyDrive/Thesis/rag_evaluation/financebench150/eval.json')

    except ValueError as e:
      print(f"Skipping question {i} due to error: {e}")
      continue

  0%|          | 0/150 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =     410.95 ms
llama_print_timings:      sample time =      59.68 ms /    89 runs   (    0.67 ms per token,  1491.39 tokens per second)
llama_print_timings: prompt eval time =    2145.81 ms /   381 tokens (    5.63 ms per token,   177.55 tokens per second)
llama_print_timings:        eval time =    3570.59 ms /    88 runs   (   40.57 ms per token,    24.65 tokens per second)
llama_print_timings:       total time =    5866.81 ms /   469 tokens
  1%|          | 1/150 [00:18<46:29, 18.72s/it]

Skipping question 12 due to error: Requested tokens (6834) exceed context window of 4096


Llama.generate: prefix-match hit

llama_print_timings:        load time =     410.95 ms
llama_print_timings:      sample time =      49.30 ms /    89 runs   (    0.55 ms per token,  1805.20 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =    3580.26 ms /    89 runs   (   40.23 ms per token,    24.86 tokens per second)
llama_print_timings:       total time =    3691.00 ms /    89 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     410.95 ms
llama_print_timings:      sample time =     164.50 ms /   256 runs   (    0.64 ms per token,  1556.27 tokens per second)
llama_print_timings: prompt eval time =   17832.84 ms /  2768 tokens (    6.44 ms per token,   155.22 tokens per second)
llama_print_timings:        eval time =   13593.84 ms /   256 runs   (   53.10 ms per token,    18.83 tokens per second)
llama_print_timings:       to

Skipping question 10 due to error: Requested tokens (8515) exceed context window of 4096


Llama.generate: prefix-match hit

llama_print_timings:        load time =     410.95 ms
llama_print_timings:      sample time =      55.13 ms /    89 runs   (    0.62 ms per token,  1614.48 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =    3482.79 ms /    89 runs   (   39.13 ms per token,    25.55 tokens per second)
llama_print_timings:       total time =    3601.06 ms /    89 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     410.95 ms
llama_print_timings:      sample time =     151.86 ms /   256 runs   (    0.59 ms per token,  1685.80 tokens per second)
llama_print_timings: prompt eval time =   22837.67 ms /  3398 tokens (    6.72 ms per token,   148.79 tokens per second)
llama_print_timings:        eval time =   14547.97 ms /   255 runs   (   57.05 ms per token,    17.53 tokens per second)
llama_print_timings:       to

...

Skipping question 147 due to error: Texts list is empty


Llama.generate: prefix-match hit

llama_print_timings:        load time =     410.95 ms
llama_print_timings:      sample time =      39.33 ms /    62 runs   (    0.63 ms per token,  1576.36 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =    2494.84 ms /    62 runs   (   40.24 ms per token,    24.85 tokens per second)
llama_print_timings:       total time =    2582.23 ms /    62 tokens
 99%|█████████▉| 149/150 [1:31:50<00:12, 12.01s/it]

Skipping question 148 due to error: Texts list is empty


Llama.generate: prefix-match hit

llama_print_timings:        load time =     410.95 ms
llama_print_timings:      sample time =      39.64 ms /    62 runs   (    0.64 ms per token,  1564.23 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =    2497.62 ms /    62 runs   (   40.28 ms per token,    24.82 tokens per second)
llama_print_timings:       total time =    2591.59 ms /    62 tokens
100%|██████████| 150/150 [1:31:54<00:00, 36.76s/it]

Skipping question 149 due to error: Texts list is empty
